In [57]:
import pandas as pd
import glob
import os

path_to_year = "../data/xls_data/20234/"

In [58]:
    
def parse_dataframe(path: str) -> pd.DataFrame:
    
    df = pd.read_html(path)
    # Concatenate all dataframes in the list along the columns axis, using inner join to handle missing data
    parsed_df = pd.concat(df, axis=1, join='inner')
    # Set the column names to the values in the second row
    parsed_df.columns = parsed_df.iloc[1]
    # Drop the first two rows as they are not data
    parsed_df = parsed_df.iloc[2:]
    # Remove the last column as it's not needed
    parsed_df = parsed_df.iloc[:, :-1]
    # Set the column names to the values in the first row
    parsed_df.columns = parsed_df.iloc[0]
    # Drop the first row as it's now redundant
    parsed_df = parsed_df.iloc[1:]
    # Prepend 'Index' to the column names to create a clear index column
    parsed_df.columns = ["Index"] + parsed_df.columns.tolist()[1:]
    # Set 'Index' as the index of the dataframe
    parsed_df.set_index("Index", inplace=True)
    # Transpose the dataframe to change the orientation of the data
    parsed_df = parsed_df.T
    # Rename a column to correct a typo in the name
    parsed_df.rename(columns={"ï¿½ Gesamtpunktzahl": "Gesamtpunktzahl"}, inplace=True)
    # Find the index of a specific column to use as a reference
    last_index = parsed_df.columns.get_loc("Note 6 in Prozent")
    # Extract column names after the reference column to use as modules
    modules = parsed_df.columns.tolist()[last_index + 1:]
    # Select only columns up to and including the reference column
    parsed_df = parsed_df.iloc[:, :last_index+1]
    # Add a new column 'modules' with the same list of modules for each row
    parsed_df["modules"] = [modules] * len(parsed_df)
    # Concatenate the new dataframe to the full dataframe, using 'Standort' as the index
    parsed_df["Standort"] = parsed_df.index
    return parsed_df

In [59]:

all_berufe = glob.glob(path_to_year + "*")


for path_to_beruf in all_berufe:
    full_df = pd.DataFrame()
    all_standorte_xls = glob.glob(path_to_beruf + "/*.xls")
    
    for path in all_standorte_xls:
        parsed_df = parse_dataframe(path)
        full_df = pd.concat([full_df, parsed_df], ignore_index=True)

    full_df = full_df.drop_duplicates(subset=full_df.columns.difference(['modules']))
    full_df["Standort"] = full_df["Standort"].str.replace("IHK ", "").str.replace("zu", "").str.replace(" ", "").str.replace("für", "")
        
    full_df.set_index("Standort", inplace=True)
    full_df.sort_index(inplace=True)
    full_df = full_df.fillna(0)
    
    # year, beruf_name = path_to_beruf.split("/")[:-2]
    year, beruf_name = path_to_beruf.split("/")[-2:]
    print(f"{year}/{beruf_name}")
    
    output_folder = f"../data/csv_data/{year}/"
    if not os.path.exists(output_folder):
        print(f"Creating folder {output_folder}")
        os.makedirs(output_folder)

    output_path = f"{output_folder}/{beruf_name}.csv"
    full_df.to_csv(output_path)
    print(f"Saved to {output_path}")

20234/Kaufmann-Kauffrau-im-E-Commerce
Creating folder ../data/csv_data/20234/
Saved to ../data/csv_data/20234//Kaufmann-Kauffrau-im-E-Commerce.csv
20234/FachinformatikerFachinformatikerin-Fachrichtung:-Digitale-Vernetzung
Saved to ../data/csv_data/20234//FachinformatikerFachinformatikerin-Fachrichtung:-Digitale-Vernetzung.csv
20234/FachinformatikerFachinformatikerin-Fachrichtung:-Anwendungsentwicklung
Saved to ../data/csv_data/20234//FachinformatikerFachinformatikerin-Fachrichtung:-Anwendungsentwicklung.csv
20234/Kaufmann-Kauffrau-für-Büromanagement
Saved to ../data/csv_data/20234//Kaufmann-Kauffrau-für-Büromanagement.csv
20234/FachinformatikerFachinformatikerin-Fachrichtung:-Systemintegration
Saved to ../data/csv_data/20234//FachinformatikerFachinformatikerin-Fachrichtung:-Systemintegration.csv
20234/FachinformatikerFachinformatikerin-Fachrichtung:-Daten-und-Prozessanalyse
Saved to ../data/csv_data/20234//FachinformatikerFachinformatikerin-Fachrichtung:-Daten-und-Prozessanalyse.csv
20

Index,Anzahl Teilnehmer,davon bestanden,Bestehensquote,ø Gesamtpunktzahl,Note 1,Note 1 in Prozent,Note 2,Note 2 in Prozent,Note 3,Note 3 in Prozent,Note 4,Note 4 in Prozent,Note 5,Note 5 in Prozent,Note 6,Note 6 in Prozent,modules
Standort,,,,,,,,,,,,,,,,,
Aachen,30,28,933,66,0,00,4,133,12,400,13,433,1,33,0,00,[Sortimentsbewirtschaftung und Vertragsanbahnu...
ArnsbergHellweg-Sauerland,9,9,1000,68,0,00,0,00,6,667,3,333,0,00,0,00,[Sortimentsbewirtschaftung und Vertragsanbahnu...
Aschaffenburg,4,4,1000,65,0,00,0,00,2,500,2,500,0,00,0,00,[Sortimentsbewirtschaftung und Vertragsanbahnu...
Berlin,65,56,862,65,0,00,9,138,27,415,23,354,4,62,2,31,[Sortimentsbewirtschaftung und Vertragsanbahnu...
Bodensee-Oberschwaben,5,5,1000,75,0,00,1,200,3,600,1,200,0,00,0,00,[Sortimentsbewirtschaftung und Vertragsanbahnu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wiesbaden,4,3,750,66,0,00,1,250,2,500,0,00,1,250,0,00,[Sortimentsbewirtschaftung und Vertragsanbahnu...
Wuppertal-Solingen-Remscheid,20,17,850,64,0,00,0,00,9,450,9,450,2,100,0,00,[Sortimentsbewirtschaftung und Vertragsanbahnu...
Würzburg-Schweinfurt,25,24,960,70,0,00,5,200,11,440,8,320,1,40,0,00,[Sortimentsbewirtschaftung und Vertragsanbahnu...


Index,Anzahl Teilnehmer,davon bestanden,Bestehensquote,ø Gesamtpunktzahl,Note 1,Note 1 in Prozent,Note 2,Note 2 in Prozent,Note 3,Note 3 in Prozent,Note 4,Note 4 in Prozent,Note 5,Note 5 in Prozent,Note 6,Note 6 in Prozent,modules,Standort
0,4,3,750,66,0,00,1,250,2,500,0,00,1,250,0,00,[Sortimentsbewirtschaftung und Vertragsanbahnu...,Wiesbaden
1,1334,1232,924,69,12,09,215,161,600,450,429,322,68,51,10,07,[Sortimentsbewirtschaftung und Vertragsanbahnu...,bundesweit
2,9,9,1000,75,0,00,1,111,7,778,1,111,0,00,0,00,[Sortimentsbewirtschaftung und Vertragsanbahnu...,Braunschweig
3,21,20,952,69,0,00,1,48,14,667,5,238,1,48,0,00,[Sortimentsbewirtschaftung und Vertragsanbahnu...,Schwaben
4,7,6,857,64,0,00,1,143,2,286,3,429,1,143,0,00,[Sortimentsbewirtschaftung und Vertragsanbahnu...,Erfurt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,8,8,1000,75,0,00,3,375,3,375,2,250,0,00,0,00,[Sortimentsbewirtschaftung und Vertragsanbahnu...,Flensburg
77,12,12,1000,80,1,83,6,500,4,333,1,83,0,00,0,00,[Sortimentsbewirtschaftung und Vertragsanbahnu...,Heilbronn-Franken
78,10,10,1000,70,1,100,2,200,2,200,5,500,0,00,0,00,[Sortimentsbewirtschaftung und Vertragsanbahnu...,Rheinhessen
79,31,31,1000,73,0,00,8,258,15,484,8,258,0,00,0,00,[Sortimentsbewirtschaftung und Vertragsanbahnu...,Kassel-Marburg
